# Heiko's Challenge

Given and ODE that describes population level dynamics, design an ABM that has the same dynamics. The definition of same dynamics is $V(t) = P$ solve for $t$ within 20%. 

The given equation is

$$
  \frac{dV}{dt} = \lambda V \left(1-\frac{V}{k}\right)
$$

- $V_0 = 1$
- $\lambda = ln 2$
- $k = 1000$


In [1]:
using DifferentialEquations

ArgumentError: ArgumentError: Package DifferentialEquations not found in current path:
- Run `import Pkg; Pkg.add("DifferentialEquations")` to install the DifferentialEquations package.


In [25]:
function solve(f, param, initial, tdomain)
    tsamples = 100
    tstart = first(tdomain)
    tend = last(tdomain)
    tlen = tend - tstart
    dt = tlen/(tsamples-1)
    tset = tstart:dt:tend
    @show tset
    sol = [initial]
    for t in 0:dt:tend
        u = sol[end] + dt*f(sol[end], t, param)
        push!(sol, u)
    end
    return sol
end

solve (generic function with 1 method)

In [24]:
V₀ = 1.0
λ = log(2)
k = 1000
f(u,t, p) = begin
    λ = first(p)
    k = last(p)
    λ*u*(1-(u/k))
end

tdomain = (0, 21)

(0, 21)

In [47]:
sol = solve(f, (λ, k), V₀, tdomain)

tset = 0.0:0.21212121212121213:21.0


101-element Array{Float64,1}:
   1.0               
   1.1468841888986576
   1.3153185739908013
   1.5084570955029775
   1.7299128216190918
   1.983824009091136 
   2.2749294237983895
   2.6086541414191298
   2.991207184009446 
   3.4296924904646002
   3.9322348645189606
   4.508122688461221 
   5.167969327283415 
   ⋮                 
 997.8393008179573   
 998.1563046219674   
 998.426885612544    
 998.6578186837932   
 998.8548963708434   
 999.0230695581017   
 999.166568507365    
 999.2890068275431   
 999.3934706952435   
 999.4825953494383   
 999.5586306251345   
 999.6234970601441   

In [48]:
yet = false
for (i, u) in enumerate(sol)
    t = (tdomain[2]-tdomain[1])i/length(sol)
    if u > 800 && ! yet
        println("$t:\t", u)
        yet = true
    end
end

12.475247524752476:	802.4081057785716


In [61]:
using Plots

## ABM Version

Deterministic ABM that has the same dynamics as the ODE logistic growth

In [62]:
function transition!(agents, param)
    V = length(agents)
    λ, k = first(param), last(param)
    dV = ceil(λ*V*(1-(V/k)))
    append!(agents, [:V for i in 0:dV])
    return agents
end


transition! (generic function with 1 method)

In [63]:
agents = [:V]
day = 0
while length(agents) < 999
    println(day, ":\t", length(agents))
    agents = transition!(agents, (λ, k))
    day += 1
end

0:	1
1:	3
2:	7
3:	13
4:	23
5:	40
6:	68
7:	113
8:	184
9:	290
10:	434
11:	606
12:	773
13:	896
14:	962
15:	989
16:	998


## Stochastic ABM 

This version is a true ABM, it models the agents as cells that divide instantaneously.

In [139]:
function transition_stoch!(agents, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = (λ*V*(1-(V/k)))*step
    p = EdV/V
    for i in 1:length(agents)
        add = rand(Float64) < p
        if add
            push!(agents, :V)
        end        
    end
    return agents
end

transition_stoch! (generic function with 2 methods)

In [148]:
agents = [:V]
day = 0
dt = 1/2
yet = false
for i in 1:ceil(30/dt)
    println(day, ":\t", length(agents))
    agents = transition_stoch!(agents, (λ, k), dt)
    day += dt
    if length(agents) > 800 && !false
        println("============\n$day:\t", length(agents), "\n================")
        yet = true
    end
    if length(agents) >= k
        break
    end
end

0:	1
0.5:	1
1.0:	2
1.5:	2
2.0:	2
2.5:	2
3.0:	2
3.5:	3
4.0:	3
4.5:	5
5.0:	7
5.5:	9
6.0:	13
6.5:	19
7.0:	27
7.5:	36
8.0:	42
8.5:	54
9.0:	72
9.5:	98
10.0:	128
10.5:	163
11.0:	202
11.5:	262
12.0:	332
12.5:	414
13.0:	490
13.5:	571
14.0:	657
14.5:	741
15.0:	801
15.0:	801
15.5:	863
15.5:	863
16.0:	896
16.0:	896
16.5:	936
16.5:	936
17.0:	956
17.0:	956
17.5:	968
17.5:	968
18.0:	981
18.0:	981
18.5:	985
18.5:	985
19.0:	989
19.0:	989
19.5:	992
19.5:	992
20.0:	995
20.0:	995
20.5:	995
20.5:	995
21.0:	1001


$$\frac{dv}{dt} = λV\log\left(\frac{k}{v}\right)$$

In [155]:
f(u, t, p) = begin
    λ, k = p
    V = u
    λ*V*log(k/V)
end

f (generic function with 3 methods)

In [156]:
sol = solve(f, (λ, k), V₀, tdomain)

tset = 0.0:0.21212121212121213:21.0


101-element Array{Float64,1}:
   1.0               
   2.0156556869506614
   3.85513289828604  
   7.005743932159319 
  12.115912516413033 
  19.977727216474033 
  31.471979427712903 
  47.476445813911894 
  68.74973032659722  
  95.81265655682066  
 128.85280938170206  
 167.67350645981645  
 211.69769996668276  
   ⋮                 
 999.9914953822414   
 999.9927458212497   
 999.9938124081337   
 999.9947221745008   
 999.9954981775757   
 999.9961600845296   
 999.9967246709024   
 999.9972062457472   
 999.9976170142698   
 999.9979673871518   
 999.9982662443952   
 999.9985211603762   

In [162]:
function transition_stoch!(agents, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = (λ*V*(log(k/V)))*step
    p = EdV/V
    for i in 1:length(agents)
        add = rand(Float64) < p
        if add
            push!(agents, :V)
        end        
    end
    return agents
end
agents = [:V]
day = 0
dt = 1/8
yet = false
for i in 1:ceil(30/dt)
    println(day, ":\t", length(agents))
    agents = transition_stoch!(agents, (λ, k), dt)
    day += dt
    if length(agents) > 800 && !false
        println("============\n$day:\t", length(agents), "\n================")
        yet = true
        break
    end
    if length(agents) >= k
        break
    end
end

0:	1
0.125:	1
0.25:	2
0.375:	2
0.5:	2
0.625:	3
0.75:	4
0.875:	6
1.0:	9
1.125:	11
1.25:	18
1.375:	25
1.5:	35
1.625:	48
1.75:	65
1.875:	78
2.0:	104
2.125:	119
2.25:	134
2.375:	154
2.5:	181
2.625:	211
2.75:	230
2.875:	257
3.0:	291
3.125:	320
3.25:	347
3.375:	378
3.5:	404
3.625:	431
3.75:	456
3.875:	498
4.0:	532
4.125:	563
4.25:	596
4.375:	622
4.5:	649
4.625:	672
4.75:	689
4.875:	711
5.0:	732
5.125:	750
5.25:	764
5.375:	782
5.5:	801


In [163]:
yet = false
for (i, u) in enumerate(sol)
    t = (tdomain[2]-tdomain[1])i/length(sol)
    if u > 800 && ! yet
        println("$t:\t", u)
        yet = true
    end
end

5.405940594059406:	802.7945435374461


In [164]:
using SemanticModels

┌ Info: Recompiling stale cache file /Users/jpf/.julia/compiled/v1.0/SemanticModels/Di9ju.ji for SemanticModels [f5ac2a72-33c7-5caf-b863-f02fefdcf428]
└ @ Base loading.jl:1190


In [165]:
using SemanticModels.ModelTools

In [166]:
ExpODEModel = ModelTools.ExpODEModels.ExpODEModel

SemanticModels.ModelTools.ExpODEModels.ExpODEModel

In [198]:
code = quote module Gompertz
    using DifferentialEquations
function f(du, u,t, p)
    λ = first(p)
    k = last(p)
    return λ*u*(1-(u/k))
end
function main()
V₀ = 1.0
λ = log(2)
k = 1000
p = (λ, k)


tdomain = (0, 21)
prob = ODEProblem(f, p, V₀, tdomain)
sol = solve(prob)
end
end
end
m = ModelTools.model(ExpODEModel, code.args[2])

matches = Expr[:(ODEProblem(f, p, V₀, tdomain))]


ExpODEModel(
  calls=Expr[:(ODEProblem(f, p, V₀, tdomain))],
  funcs=Expr[:(function f(du, u, t, p)
      #= In[198]:4 =#
      λ = first(p)
      #= In[198]:5 =#
      k = last(p)
      #= In[198]:6 =#
      return λ * u * (1 - u / k)
  end)],
  variables=NamedTuple{(:state, :flux, :params),Tuple{Array{Expr,1},Array{Expr,1},Array{Expr,1}}}[(state = [], flux = [], params = [])],
  domains=Array{Expr,1}[[:(V₀ = 1.0)]],  values=Array{Expr,1}[[:(p = (λ, k))]]
)

In [207]:
using ModelingToolkit

In [205]:
h(x) = head(x)
h(x::Symbol) = x

@variables u λ k
ModelTools.Transformations.postwalk(m.funcs[1]) do x
    try
        if isa(x, Expr) && h(x) == :return
            println(x, " ",h(x))
            @show factor = x.args[end].args[end]
            
        else
            if isa(x, Symbol)
                @variables 
            end
    finally
        x
    end
    x
end

return λ * u * (1 - u / k) return
factor = (x.args[end]).args[end] = :(1 - u / k)


:(function f(du, u, t, p)
      #= In[198]:4 =#
      λ = first(p)
      #= In[198]:5 =#
      k = last(p)
      #= In[198]:6 =#
      return λ * u * (1 - u / k)
  end)

In [197]:
m.funcs[1]

:(function f(du, u, t, p)
      #= In[195]:4 =#
      λ = first(p)
      #= In[195]:5 =#
      k = last(p)
      #= In[195]:6 =#
      return λ * u * (1 - u / k)
  end)

In [210]:
@variables V λ k u t

(V(), λ(), k(), u(), t())

In [212]:
dvdt = f(V, t, (λ, k))

(λ() * V()) * log(k() / V())

In [213]:
dump(dvdt)


Operation
  op: * (function of type typeof(*))
  args: Array{Expression}((2,)) Expression[λ() * V(), log(k() / V())]


In [216]:
ModelingToolkit.vars(dvdt/V)

Set(Variable[k, V, λ])

In [218]:
dvdt/V

((λ() * V()) * log(k() / V())) / V()

In [239]:
# import Base: rand
# ModelingToolkit.@register rand(x)
# Operation(ifelse, [rand(V) < dvdt/V, 1, 0])
ModelingToolkit.@register Bernoulli(p)
Bernoulli(p) = ifelse(rand() < p, 0, 1)

Bernoulli (generic function with 2 methods)

In [240]:
Bernoulli(dvdt/V)

Bernoulli(((λ() * V()) * log(k() / V())) / V())

In [245]:
function transition_stoch!(agents, dudt, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = dudt*step
    p = EdV/V
    add = Bernoulli(p)
    @show add
    for i in 1:length(agents)
        if @show(add, (V=length(agents), λ=first(param), k=last(param)))
            push!(agents, :V)
        end
    end
    return agents
end

transition_stoch! (generic function with 3 methods)

In [246]:
@variables abmdt

(abmdt(),)

In [247]:
transition_stoch!(agents, dvdt, (λ, k), abmdt)

add = Bernoulli((((λ() * V()) * log(k() / V())) * abmdt()) / 801)
add = Bernoulli((((λ() * V()) * log(k() / V())) * abmdt()) / 801)
(V = length(agents), λ = first(param), k = last(param)) = (V = 801, λ = λ(), k = k())


TypeError: TypeError: non-boolean (NamedTuple{(:V, :λ, :k),Tuple{Int64,Operation,Operation}}) used in boolean context